In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from googleapiclient.discovery import build

# Set up the Google Custom Search API
api_key = 'A***************************TU-4'
search_engine_id = "c**************5ba"

# Search for the LinkedIn profile using the company name
def search_linkedin_profile(company_name):
    service = build("customsearch", "v1", developerKey=api_key)
    query = f'{company_name} site:linkedin.com'
    result = service.cse().list(q=query, cx=search_engine_id).execute()
    items = result.get('items', [])
    for item in items:
        link = item.get('link', '')
        if 'linkedin.com/company/' in link:
            return link

num_pages = 4  # Set the number of pages you want to scrape
increment = 15  # Set the increment value for each page

data_list = []

for page in range(1, num_pages + 1):
    page_number = (page - 1) * increment
    url = f"https://www.houzz.co.uk/professionals/interior-designers/p/{page_number}"
    
    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object with the response content
    soup = BeautifulSoup(response.content, "html.parser")

    script_tags = soup.find_all('script')

    for script_tag in script_tags:
        if script_tag.has_attr('type') and script_tag['type'] == 'application/ld+json':
            # Parse the JSON data
            json_data = script_tag.string
            try:
                data = json.loads(json_data)

                # Get the values of "name", "telephone", and "address" keys
                name = data.get('name')
                telephone = data.get('telephone')
                # Get the values of 'streetAddress', 'addressLocality', 'addressRegion', 'postalCode', and 'addressCountry'
                street_address = data.get('address', {}).get('streetAddress', '')
                locality = data.get('address', {}).get('addressLocality', '')
                region = data.get('address', {}).get('addressRegion', '')
                postal_code = data.get('address', {}).get('postalCode', '')
                country = data.get('address', {}).get('addressCountry', '')

                # Concatenate the values with commas
                address = ', '.join([street_address, locality, region, postal_code, country])

                company_name = data.get('name')

                # Search for the LinkedIn profile
                linkedin_profile = search_linkedin_profile(company_name)

                if linkedin_profile:
                    linkedin_profile += '/about/'

                data_list.append({'Name': company_name, 'Telephone': data.get('telephone'), 'Address': address, 'LinkedIn URL': linkedin_profile})
            except json.JSONDecodeError:
                print("Error decoding JSON data.")

# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

# Drop rows with "second-to-none" or None in "LinkedIn URL" column
df.drop(df[(df["LinkedIn URL"] == "https://www.linkedin.com/company/second-to-none-inc/about/") | (df["LinkedIn URL"].isnull())].index, inplace=True)

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)
# Print the DataFrame
print(df["LinkedIn URL"])


In [ ]:
import os
import random
import sys
import time
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from seleniumwire import webdriver as wired_webdriver
from bs4 import BeautifulSoup
import pandas as pd
    
    
if __name__ == '__main__':
    email = "pr********************m"
    password = "************k"

    options = webdriver.ChromeOptions()
        #options.add_argument('proxy-server=106.122.8.54:3128')
    options.add_argument(r'--user-data-dir=C:\Users\Priyank\AppData\Local\Google\Chrome\Default')
    
    browser = uc.Chrome(options=options)
    
    
    def scrape_linkedin(url):
        
        browser.get(url)

        SCROLL_PAUSE_TIME = 5
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")

        for i in range(3):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        src = browser.page_source
        soup = BeautifulSoup(src, 'html.parser')

        web = ''
        comp = ''

        dt_tags = soup.find_all('dt', class_='mb1 text-heading-medium')

        for dt_tag in dt_tags:
            dt_text = dt_tag.text.strip()
            if dt_text == 'Website':
                dd_tag = dt_tag.find_next_sibling('dd')
                web = dd_tag.text.strip()
            elif dt_text == 'Company size':
                dd_tag = dt_tag.find_next_sibling('dd')
                comp = dd_tag.text.strip()

        return web, comp


# Create new columns to store the results
df['Website'] = ''
df['Company Size'] = ''

# Iterate through each row
for index, row in df.iterrows():
    linkedin_url = row['LinkedIn URL']

    # Skip row if LinkedIn URL is None
    if linkedin_url is None:
        continue

    # Scrape LinkedIn page and get results
    website, company_size = scrape_linkedin(linkedin_url)

    # Store the results in the DataFrame
    df.at[index, 'Website'] = website
    df.at[index, 'Company Size'] = company_size

    # Pause for a few seconds before scraping the next URL
    time.sleep(3)

# Print the updated DataFrame
df

In [ ]:
# Assuming your DataFrame is named df
df['Email'] = ""
for index, row in df.iterrows():
    website = row["Website"]
    if website:
        try:
            response = requests.get(website)
            soup = BeautifulSoup(response.text, "html.parser")
            email = soup.select_one("a[href^='mailto:']")
            if email:
                email_id = email["href"].replace("mailto:", "")
                df.at[index, "Email"] = email_id
        except requests.exceptions.RequestException as e:
           continue
df

In [ ]:
# Convert the values to strings
df = df.dropna(subset=['Company Size','min_employees','max_employees'])

df['Company Size'] = df['Company Size'].astype(str)

# Trim the data column and remove 'employees'
df['Company Size'] = df['Company Size'].str.replace(' employees', '')

# Split the data column into separate columns using the '-' separator
split_data = df['Company Size'].str.split('-', expand=True)


# Extract the minimum and maximum employee values separately
df['min_employees'] = split_data[0].astype(int)
df['max_employees'] = split_data[1].astype(int)


df['max_employees'] = df['max_employees'].str.replace(',', '')
df = df[df['max_employees'] >= 30]
df



In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
import google.auth
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
from time import sleep as wait

SERVICE_ACCOUNT_FILE = 'scrapper.json'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
# creds = None
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='Kosan!A3:AF').execute()
# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1GWkZibZY2UIjaqeyUWbB_NIlSMBrgz-ZXkAkGVA7I6c'
service = build('sheets', 'v4', credentials=creds)
sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range='Kosan!A3:AF').execute()


# sending to google sheets through API
request = sheet.values().append(spreadsheetId=SAMPLE_SPREADSHEET_ID, range='Kosan!A3:AF',
                                    valueInputOption='USER_ENTERED', body={'values': df.values.tolist()}).execute()
print(f'Successfully Scrapped')
wait(0.45)